---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head()

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    
    return len(spam_data[spam_data['target'] == 1])/len(spam_data)*100

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    vect = CountVectorizer().fit(X_train)    
    
    return sorted(vect.get_feature_names(), key=lambda x: len(x), reverse=True)[0]

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    vect = CountVectorizer().fit(X_train)
    
    X_train_v = vect.transform(X_train)
    X_test_v = vect.transform(X_test)
    
    
    mnbc = MultinomialNB(alpha=0.1).fit(X_train_v, y_train)
    preds = mnbc.predict(X_test_v)
    
    return roc_auc_score(y_test, preds)

In [8]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    vect = TfidfVectorizer().fit(X_train)
    
    X_train_v = vect.transform(X_train)
    
    tfidfs =  X_train_v.max(0).toarray()[0]
    names = np.array(vect.get_feature_names())
    
    res = list(zip(names, tfidfs))
    res.sort(key=lambda x: x[1])
    
    small = res[:20]
    large = res[:-21:-1]
    large.sort()
    large.sort(key=lambda x: x[1], reverse=True)
    
    small = pd.Series([el[1] for el in small], index=[el[0] for el in small])
    large = pd.Series([el[1] for el in large], index=[el[0] for el in large])    
    
    return small,large

In [10]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 afternoons      0.091250
 approaching     0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
def answer_five():
    
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_tr_v = vect.transform(X_train)
    
    mnbc = MultinomialNB(alpha=0.1).fit(X_tr_v, y_train)
    preds = mnbc.predict(vect.transform(X_test))    
    
    return roc_auc_score(y_test, preds)

In [12]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():
    
    spam_data['len'] = spam_data['text'].str.len()
    
    spam = spam_data[spam_data['target'] == 1]
    nspam = spam_data[spam_data['target'] == 0]
    
    
    return nspam['len'].mean(), spam['len'].mean()

In [14]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC

def answer_seven():
    
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_tr_v = vect.transform(X_train)
    
    X_tr = add_feature(X_tr_v, X_train.str.len())
    X_te = add_feature(vect.transform(X_test), X_test.str.len())
    
    svc = SVC(C=10000).fit(X_tr, y_train)
    preds = svc.predict(X_te)
    
    return roc_auc_score(y_test, preds)

In [17]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
def answer_eight():
    
    spam_data['digits'] = spam_data['text'].str.replace(r'(\D+)', '')
    spam_data['digits'] = np.where(spam_data['digits'], spam_data['digits'].str.len(), 0)
    
    spam = spam_data[spam_data['target'] == 1]
    nspam = spam_data[spam_data['target'] == 0]
    
    return nspam['digits'].mean(), spam['digits'].mean()

In [19]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [69]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    vect = TfidfVectorizer(min_df=5, ngram_range=(1, 3)).fit(X_train)
    X_tr_v = vect.transform(X_train)
    
    digits_tr = X_train.str.replace(r'\D+', '')
    digits_tr = np.where(digits_tr, digits_tr.str.len(), 0)
    digits_te = X_test.str.replace(r'\D+', '')
    digits_te = np.where(digits_te, digits_te.str.len(), 0)
    
    X_tr = add_feature(X_tr_v, [X_train.str.len(), digits_tr])
    X_te = add_feature(vect.transform(X_test), [X_test.str.len(), digits_te])
    
    lr = LogisticRegression(C=100).fit(X_tr, y_train)
    preds = lr.predict(X_te)    
    
    return roc_auc_score(y_test, preds)

In [70]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [97]:
def answer_ten():
    
    spam = spam_data[spam_data['target'] == 1]['text'].str.replace(r'\w', '')
    spam = np.where(spam, spam.str.len(), 0)
    
    nspam = spam_data[spam_data['target'] == 0]['text'].str.replace(r'\w', '')
    nspam = np.where(nspam, nspam.str.len(), 0)
    
    return nspam.mean(), spam.mean()

In [98]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [177]:
def answer_eleven():
    
    vect = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=(2, 5)).fit(X_train)
    X_tr_v = vect.transform(X_train)
    X_te_v = vect.transform(X_test)
    
    len_tr = X_train.str.len()
    dig_tr = X_train.str.replace(r'\D', '').str.len()
    nwc_tr = X_train.str.replace(r'\w', '').str.len()
    
    len_te = X_test.str.len()
    dig_te = X_test.str.replace(r'\D', '').str.len()
    nwc_te = X_test.str.replace(r'\w', '').str.len()
    
    X_tr = add_feature(X_tr_v, [len_tr, dig_tr, nwc_tr])
    X_te = add_feature(X_te_v, [len_te, dig_te, nwc_te])
    
    lr = LogisticRegression(C=100).fit(X_tr, y_train)
    preds = lr.predict(X_te)
    
    score = roc_auc_score(y_test, preds)
    
    names = np.array(vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])

    coefs = lr.coef_[0].argsort()
    
    small = names[coefs[:10]]
    large = names[coefs[:-11:-1]]
    
    return score, small, large

In [178]:
answer_eleven()

(0.97885931107074342,
 array(['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'], 
       dtype='<U19'),
 array(['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww',
        'ar'], 
       dtype='<U19'))